# Reappraisal Training on PyTorch Lightning

## Setup
- When running on Google Colab, mount Google Drive to access scripts.
- `cd` into the project root and install dependencies:

In [1]:
%load_ext autoreload

# import nltk
# nltk.download('punkt')

In [2]:
%autoreload
import torch

# Define constants
STRAT = 'obj'
BATCH_SIZE = 64 

In [9]:
torch.cuda.device_count()

0

2021-03-04 10:26:01,805	ERROR worker.py:1109 -- listen_error_messages_raylet: Connection closed by server.
2021-03-04 10:26:01,809	ERROR worker.py:919 -- print_logs: Connection closed by server.
2021-03-04 10:26:01,809	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.


## Load LDH Data

Contains the following:

- LDHI
- LDHII

In [3]:
import os
from reappraisalmodel.ldhdata import LDHDataModule

ROOT_DIR = os.getcwd()
ldhdata = LDHDataModule(data_dir=ROOT_DIR, strat=STRAT)
ldhdata.load_train_data()
ldhdata.load_eval_data()

Loading cached processed dataset at /home/ubuntu/reapp/output/training/obj/cache-8217e0ac99d256e6.arrow


Training data loaded from disk.
Encoding Training Data:
Evaluation data loaded from disk.
Encoding Test Data


## Run K-Fold Training


In [9]:
%autoreload 2
from reappraisalmodel.trainers import kfold_train

results = kfold_train(1, ldhdata, strat=STRAT, 
        max_epochs=1,
        limit_train_batches=1,
        limit_val_batches=1,
        s3_bucket=False,
)


Created temporary directory: /tmp/tmpq71exrki


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Training on split 0


/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory reapp_logs/01foldCV_obj_20210304_091945/split_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)
/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Successful 0_epoch=00-val_loss=6.23.ckpt to s3: True
{'val_loss': 6.233573913574219, 'train_loss': 23.186851501464844, 'num_epochs': 0, 'r2score': tensor(-0.5544), 'explained_var': tensor(-0.3108)}
       val_loss  train_loss  num_epochs   r2score  explained_var
count  1.000000    1.000000         1.0  1.000000        1.00000
mean   6.233574   23.186852         0.0 -0.554429       -0.31076
std         NaN         NaN         NaN       NaN            NaN
min    6.233574   23.186852         0.0 -0.554429       -0.31076
25%    6.233574   23.186852         0.0 -0.554429       -0.31076
50%    6.233574   23.186852         0.0 -0.554429       -0.31076
75%    6.233574   23.186852         0.0 -0.554429       -0.31076
max    6.233574   23.186852         0.0 -0.554429       -0.31076


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df['r2score'] = df['r2score'].apply(lambda x: x.item())
df['explained_var'] = df['explained_var'].apply(lambda x: x.item())
df.describe()

## Tuning Hyperparameters


2021-03-04 09:48:23,292	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-04 09:48:25,036	WARNING function_runner.py:540 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Memory usage on this node: 2.1/59.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0.25/1 GPUs, 0.0/35.25 GiB heap, 0.0/12.16 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /home/ubuntu/reapp/reapp_logs/tune/_inner_2021-03-04_09-48-25
Number of trials: 1/2 (1 RUNNING)
+--------------------+----------+-------+-------------+
| Trial name         | status   | loc   |          lr |
|--------------------+----------+-------+-------------|
| _inner_c38a9_00000 | RUNNING  |       | 0.000782566 |
+--------------------+----------+-------+-------------+




(pid=17126) 2021-03-04 09:48:29,682	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=17126) Traceback (most recent call last):
(pid=17126)   File "/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=17126)     self._entrypoint()
(pid=17126)   File "/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/ray/tune/function_runner.py", line 315, in entrypoint
(pid=17126)     return self._trainable_func(self.config, self._status_reporter,
(pid=17126)   File "/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=17126)     output = fn()
(pid=17126)   File "/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/ray/tune/function_runner.py", line 651, in _inner
(pid=17126)     inner(config, checkpoint_dir=None)
(pid=17126)   File "/home/ubuntu/anaconda3/envs/reapp/lib/python3.8/site-packages/ray/tune/function_runner.py", line 645, i

Result for _inner_c38a9_00001:
  {}
  
Result for _inner_c38a9_00000:
  {}
  
== Status ==
Memory usage on this node: 3.0/59.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0.0/1 GPUs, 0.0/35.25 GiB heap, 0.0/12.16 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /home/ubuntu/reapp/reapp_logs/tune/_inner_2021-03-04_09-48-25
Number of trials: 2/2 (2 ERROR)
+--------------------+----------+-------+-------------+
| Trial name         | status   | loc   |          lr |
|--------------------+----------+-------+-------------|
| _inner_c38a9_00000 | ERROR    |       | 0.000782566 |
| _inner_c38a9_00001 | ERROR    |       | 0.0125501   |
+--------------------+----------+-------+-------------+
Number of errored trials: 2
+--------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name         |   # failures | error file           

TuneError: ('Trials did not complete', [_inner_c38a9_00000, _inner_c38a9_00001])

In [ ]:
%autoreload
import pytorch_lightning as lit

from reappraisalmodel.lightningreapp import LightningReapp

model = LightningReapp({
    'lr': 1e-3,
    'hidden_layer_size': 50
})

trainer = lit.Trainer(fast_dev_run=1)
trainer.fit(model, ldhdata)


## Extra!

In [ ]:
resp = s3.list_objects(Bucket='ldhdata')

In [ ]:
key="obj/{'metrics': {'epoch': tensor(10.), 'val_loss': tensor(1.2882), 'r2score': tensor(0.5832), 'explained_var': tensor(0.6109), 'train_loss': tensor(1.1194, device='cuda:0')}, 'checkpoint': '/tmp/tmpvsrtlvfl/reappmodel_obj_20210227_203013/2_epoch=07-val_loss=1.12.ckpt', 'num_epochs': 10}-20210227_203013-2_epoch=07-val_loss=1.12.ckpt"
s3.copy({
    'Bucket': 'ldhdata',
    'Key': key
},
Bucket='ldhdata',
Key='obj/20210227_203013-2_epoch=07-val_loss=1.12.ckpt')


In [ ]:
import torch
metrics = [
    {'epoch': torch.tensor(7.), 'val_loss': 1.1842, 'r2score': torch.tensor(0.6130), 'explained_var': torch.tensor(0.6388), 'train_loss': torch.tensor(1.1642, device='cuda:0')},
    {'epoch': torch.tensor(8.), 'val_loss': 1.1819, 'r2score': torch.tensor(0.6087), 'explained_var': torch.tensor(0.6377), 'train_loss': torch.tensor(1.0931, device='cuda:0')}, 
    {'epoch': torch.tensor(9.), 'val_loss': 1.2094, 'r2score': torch.tensor(0.5926), 'explained_var': torch.tensor(0.6366), 'train_loss': torch.tensor(1.1363, device='cuda:0')}, 
    {'epoch': torch.tensor(10.), 'val_loss': 1.2712, 'r2score': torch.tensor(0.5906), 'explained_var': torch.tensor(0.6339), 'train_loss': torch.tensor(1.0842, device='cuda:0')}, 
    {'epoch': torch.tensor(10.), 'val_loss': 1.2882, 'r2score': torch.tensor(0.5832), 'explained_var': torch.tensor(0.6109), 'train_loss': torch.tensor(1.1194, device='cuda:0')}, 
]

df = pd.DataFrame(metrics)
for key in ['r2score', 'epoch', 'explained_var', 'train_loss']:
    df[key] = df[key].apply(lambda x: x.item())

In [ ]:
upload_report = upload_file('this.csv', 'ldhdata', f'obj/20210227_203013-report.csv')
print(f"Successful Uploading Report to s3: {upload_report}")

In [ ]:
df.describe()

In [ ]:
for obj in resp['Contents']:
    print(obj['Key'])

### Tokenization

In [ ]:
# hide
# Returns a BatchEncoding of the text.
tokenized = tokenizer(text = ["This is the first test sentence!", "This is the second, better test sentence."], 
    padding='max_length', max_length=150)

for idx, sent in enumerate(tokenized.input_ids):
    print(f"Sentence            {idx}: {tokenizer.convert_ids_to_tokens(sent)}")
    print(f"Tokenized Attention {idx}: {tokenized[idx].attention_mask}")



In [ ]:
%autoreload
import torch
import pytorch_lightning as lit
from reappraisalmodel.lightningreapp import LightningReapp

default_config = default_config = {
    'lr': 1e-3,
    'hidden_lay': 50
}

model = LightningReapp(default_config)

trainer = lit.Trainer(
    gpus = 1 if torch.cuda.is_available() else None,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    max_epochs=10,
    fast_dev_run=2,
    terminate_on_nan=True)

model = LightningReapp(default_config)

trainer.fit(model, ldhdata.train_dataloader(), ldhdata.val_dataloader())



In [ ]:
model = LightningReapp.load_from_checkpoint(
    '/Users/danielpham/Google Drive/ldh/lightning_logs_obj_0223/version_2/checkpoints/epoch=1-step=337.ckpt', map_location='cpu')

model.eval()
model

In [ ]:
%autoreload 
import pandas as pd
from pytorch_lightning import Trainer 

from reappraisalmodel.lightningreapp import LightningReapp

config = {
    'lr': 1e-3,
    'num_embedding_layers': 2
}

model = LightningReapp(config)
trainer = Trainer(
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    terminate_on_nan=True)

test_dataloader = ldhdata

In [ ]:
dfs = []
for result in results:
    print(len(result['predict']))

In [ ]:
import boto3
import pickle

from sagemaker import get_execution_role
role = get_execution_role()

bucket = 'ldhdata'
file = 'Master_Final_TrainingData.csv'

s3client = boto3.client('s3')

response = s3client.get_object(Bucket=bucket, Key=file)

import codecs 
import csv

train = csv.DictReader(codecs.getreader("utf-8")(response["Body"])) # returns an ordered dict
